<a href="https://colab.research.google.com/github/anna-boser/AutoGluon_PM_paper/blob/main/ML_model_random_CV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up Autogluon and get data

In [1]:
# Uninstall mkl for faster neural-network training time
!pip uninstall -y mkl
# Upgrade pip to ensure the latest package versions are available
!pip install -U pip
# Upgrade setuptools to be compatible with namespace packages
!pip install -U setuptools wheel
!pip install -U "mxnet<2.0.0"
# Install autogluon (Tutorial based on autogluon==0.1.0)
!pip install autogluon
# Upgrade ipykernel (Necessary for Colab)
!pip install -U ipykernel

Found existing installation: mkl 2019.0
Uninstalling mkl-2019.0:
  Successfully uninstalled mkl-2019.0
     |████████████████████████████████| 2.1 MB 5.4 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.9 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 MB 12.7 MB/s eta 0:00:00
  Atte

IMPORTANT: PLEASE READ BELOW INSTRUCTIONS

MANUAL STEP: Restart Colab Runtime, then execute remaining cells

In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold
import gc
import math
import os
import glob

In [3]:
from google.colab import drive
drive.mount('/content/drive')

os.chdir('/content/drive/MyDrive/NASA PM_estimation/')

Mounted at /content/drive


In [4]:
df = pd.read_csv("Data/datasets/Train.csv")
df.head()

,Unnamed: 0,Unique,Id,Day,Lat,Lon,Elevation,Emissions,Forest,Roads,...,Plumes_Med,Plumes_Low,Max_Temp,Max_Wind,Precip,Rel_Humidity,Wind_Dir,BLH,AOD,PM
0,1,1131_1,1131,1,36.840574,-121.366314,122.986002,0.0,0,0.0,...,0,0,53.4,5.3,0.0,67.0,251.0,758.111816,NaN,3.7
1,2,1131_100,1131,100,36.840574,-121.366314,122.986002,0.0,0,0.0,...,0,0,64.1,3.6,0.0,48.0,308.0,1155.824219,NaN,2.6
2,3,1131_101,1131,101,36.840574,-121.366314,122.986002,0.0,0,0.0,...,0,0,62.7,5.1,0.0,64.0,200.0,1162.292725,NaN,2.9
3,4,1131_102,1131,102,36.840574,-121.366314,122.986002,0.0,0,0.0,...,0,0,67.7,7.9,0.0,68.0,221.0,263.273407,NaN,0.5
4,5,1131_104,1131,104,36.840574,-121.366314,122.986002,0.0,0,0.0,...,0,0,60.1,6.5,0.0,44.0,314.0,1268.919434,0.097667,2.4


# Crossvalidate 
Make tables of crossvalidated predictions by station for various models \\
\
Models chosen are those from Autogluon default: \\
\
hyperparameters = {
‘NN’: {}, ‘GBM’: [

{}, {‘extra_trees’: True, ‘AG_args’: {‘name_suffix’: ‘XT’}},

], ‘CAT’: {}, ‘RF’: [

{‘criterion’: ‘gini’, ‘AG_args’: {‘name_suffix’: ‘Gini’, ‘problem_types’: [‘binary’, ‘multiclass’]}}, {‘criterion’: ‘entropy’, ‘AG_args’: {‘name_suffix’: ‘Entr’, ‘problem_types’: [‘binary’, ‘multiclass’]}}, {‘criterion’: ‘mse’, ‘AG_args’: {‘name_suffix’: ‘MSE’, ‘problem_types’: [‘regression’]}},

], ‘XT’: [

{‘criterion’: ‘gini’, ‘AG_args’: {‘name_suffix’: ‘Gini’, ‘problem_types’: [‘binary’, ‘multiclass’]}}, {‘criterion’: ‘entropy’, ‘AG_args’: {‘name_suffix’: ‘Entr’, ‘problem_types’: [‘binary’, ‘multiclass’]}}, {‘criterion’: ‘mse’, ‘AG_args’: {‘name_suffix’: ‘MSE’, ‘problem_types’: [‘regression’]}},

], ‘KNN’: [

{‘weights’: ‘uniform’, ‘AG_args’: {‘name_suffix’: ‘Unif’}}, {‘weights’: ‘distance’, ‘AG_args’: {‘name_suffix’: ‘Dist’}},

], ‘custom’: [‘GBM’]

}

In [5]:
def cv(hyperparameters):
  df['fold'] = np.random.randint(1, 11, df.shape[0]) # 10 fold random CV
  n_fold = len(set(df['fold']))
  print(n_fold) #should be 10
  kf = GroupKFold(n_fold)
  split = kf.split(df, groups = df['fold'])

  metric_df = pd.DataFrame(columns = ['station', 'Day', 'PM', 'PM_pred', 'rmse', 'bias'])

  for i, (train_idx, test_idx) in enumerate(split):
    print(f'Starting training fold {i}.')
    _ = gc.collect()

    features = ['Day', 'Lat', 'Lon', 'Elevation', 'Emissions', 'Forest',
        'Roads', 'Streets', 'Plumes_High', 'Plumes_Med', 'Plumes_Low',
        'Max_Temp', 'Max_Wind', 'Precip', 'Rel_Humidity', 'Wind_Dir', 'BLH',
        'AOD']
    label_column = 'PM'

    X = df[features]
    y = df[label_column]

    train_data = TabularDataset(pd.concat([X.loc[train_idx], y.loc[train_idx]], axis=1))
    test_data = TabularDataset(pd.concat([X.loc[test_idx], y.loc[test_idx]], axis=1))

    predictor = TabularPredictor(label=label_column, eval_metric='r2').fit(train_data=train_data, hyperparameters=hyperparameters)

    y_test = test_data[label_column]

    test_data_nolab = test_data.drop(labels = [label_column], axis = 1)

    y_pred = predictor.predict(test_data_nolab)
    
    bias = y_pred-y_test
    rmse = np.abs(bias)

    station = np.repeat(df.loc[test_idx]['Id'].iloc[0], len(test_data['Day']))
    df_to_append = pd.DataFrame({'station': station, 'Day': test_data['Day'], 'PM':y_test, 'PM_pred': y_pred, 'rmse': rmse, 'bias':bias})

    metric_df = metric_df.append(df_to_append, ignore_index = True)

  return metric_df

In [6]:
CAT = cv({'CAT': {}})
CAT.to_csv("Data/output/CV_random/CAT.csv", index = False)

10
Starting training fold 0.


No path specified. Models will be saved in: "AutogluonModels/ag-20220424_231842/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20220424_231842/"
AutoGluon Version:  0.4.0
Python Version:     3.7.13
Operating System:   Linux
Train Data Rows:    8851
Train Data Columns: 18
Label Column: PM
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (199.1, 0.1, 8.79009, 7.75768)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    12238.44 MB
	Train Data (Original)  Memory Usage: 1.27 MB (0.0% of available memory)
	Inferring data type o

Starting training fold 1.


	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	Types of features in processed data (raw dtype, special dtypes):
		('float', [])     : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])       :  1 | ['Day']
		('int', ['bool']) :  4 | ['Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	0.1s = Fit runtime
	18 features in original data used to generate 18 features in processed data.
	Train Data (Processed) Memory Usage: 1.03 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.1s ...
AutoGluon will gauge predictive performance using evaluation metric: 'r2'
	To change this, specify the eval_metric parameter of Predictor()
Automatically generating train/validation split with holdout_frac=0.1, Train Rows: 7993, Val Rows: 889
Fitting 1 L1 models ...
Fitting m

Starting training fold 2.


...

	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	Types of features in processed data (raw dtype, special dtypes):
		('float', [])     : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])       :  1 | ['Day']
		('int', ['bool']) :  4 | ['Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	0.1s = Fit runtime
	18 features in original data used to generate 18 features in processed data.
	Train Data (Processed) Memory Usage: 1.04 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.11s ...
AutoGluon will gauge predictive performance using evaluation metric: 'r2'
	To change this, specify

Starting training fold 8.


		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	Types of features in processed data (raw dtype, special dtypes):
		('float', [])     : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])       :  1 | ['Day']
		('int', ['bool']) :  4 | ['Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	0.1s = Fit runtime
	18 features in original data used to generate 18 features in processed data.
	Train Data (Processed) Memory Usage: 1.04 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.11s ...
AutoGluon will gauge predictive performance using evaluation metric: 'r2'
	To change this, specify the eval_metric parameter of Predictor()
Automatically generating train/validation split with holdout_frac=0.1, Train Rows: 8048, Val Ro

Starting training fold 9.


	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	Types of features in processed data (raw dtype, special dtypes):
		('float', [])     : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])       :  1 | ['Day']
		('int', ['bool']) :  4 | ['Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	0.1s = Fit runtime
	18 features in original data used to generate 18 features in processed data.
	Train Data (Processed) Memory Usage: 1.04 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.11s ...
AutoGluon will gauge predictive performance using evaluation metric: 'r2'
	To change this, specify the eval_metric parameter of Predictor()
Automatically generating train/validation split with holdout_frac=0.1, Train Rows: 8049, Val Rows: 895
Fitting 1 L1 models ...
Fitting 

In [7]:
NN = cv({'NN': {}})
NN.to_csv("Data/output/CV_random/NN.csv", index = False)

No path specified. Models will be saved in: "AutogluonModels/ag-20220424_233847/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20220424_233847/"
AutoGluon Version:  0.4.0
Python Version:     3.7.13
Operating System:   Linux
Train Data Rows:    8849
Train Data Columns: 18
Label Column: PM
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (199.1, 0.1, 8.80299, 7.72722)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...


10
Starting training fold 0.


Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    12094.79 MB
	Train Data (Original)  Memory Usage: 1.27 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 4 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	Types of features in processed data (raw dtype, special dtypes):
		('float', [])     : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		(

Starting training fold 1.


	0.6572	 = Validation score   (r2)
	79.28s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	0.6572	 = Validation score   (r2)
	0.0s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 79.76s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20220424_234043/")
No path specified. Models will be saved in: "AutogluonModels/ag-20220424_234203/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20220424_234203/"
AutoGluon Version:  0.4.0
Python Version:     3.7.13
Operating System:   Linux
Train Data Rows:    8872
Train Data Columns: 18
Label Column: PM
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (199.1, 0.1, 8.84812, 7.78869)
	If 'regression' i

...

	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	Types of features in processed data (raw dtype, special dtypes):
		('float', [])     : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])       :  1 | ['Day']
		('int', ['bool']) :  4 | ['Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	0.1s = Fit runtime
	18 features in original data used to generate 18 features in processed data.
	Train Data (Processed) Memory Usage: 1.04 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.12s ...
AutoGluon will gauge predictive performance using evaluation metric: 'r2'
	To change this, specify the eval_metric parameter of Predictor()
Automatically generating train/validation split with holdout_frac=0.1, Tra

Starting training fold 8.


	0.6455	 = Validation score   (r2)
	37.52s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	0.6455	 = Validation score   (r2)
	0.0s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 37.96s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20220424_234725/")
No path specified. Models will be saved in: "AutogluonModels/ag-20220424_234803/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20220424_234803/"
AutoGluon Version:  0.4.0
Python Version:     3.7.13
Operating System:   Linux
Train Data Rows:    8959
Train Data Columns: 18
Label Column: PM
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (170.6, 0.1, 8.77485, 7.54417)
	If 'regression' i

Starting training fold 9.


	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	Types of features in processed data (raw dtype, special dtypes):
		('float', [])     : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])       :  1 | ['Day']
		('int', ['bool']) :  4 | ['Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	0.1s = Fit runtime
	18 features in original data used to generate 18 features in processed data.
	Train Data (Processed) Memory Usage: 1.04 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.1s ...
AutoGluon will gauge predictive performance using evaluation metric: 'r2'
	To change this, specify 

In [8]:
GBM = cv({'GBM': {}})
GBM.to_csv("Data/output/CV_random/GBM.csv", index = False)

No path specified. Models will be saved in: "AutogluonModels/ag-20220424_235404/"


10
Starting training fold 0.


Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20220424_235404/"
AutoGluon Version:  0.4.0
Python Version:     3.7.13
Operating System:   Linux
Train Data Rows:    8848
Train Data Columns: 18
Label Column: PM
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (199.1, 0.1, 8.82538, 7.71494)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    12465.86 MB
	Train Data (Original)  Memory Usage: 1.27 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify

Starting training fold 1.


	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 4 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	Types of features in processed data (raw dtype, special dtypes):
		('float', [])     : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])       :  1 | ['Day']
		('int', ['bool']) :  4 | ['Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	0.2s = Fit runtime
	18 features in original data used to generate 18 features in processed data.
	Train Data (Processed) Memory Usage: 1.03 MB (0.0% of available memory)
Data preprocessing

...

	0.866	 = Validation score   (r2)
	10.04s	 = Training   runtime
	0.29s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	0.866	 = Validation score   (r2)
	0.0s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 11.66s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20220424_235505/")
No path specified. Models will be saved in: "AutogluonModels/ag-20220424_235518/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20220424_235518/"
AutoGluon Version:  0.4.0
Python Version:     3.7.13
Operating System:   Linux
Train Data Rows:    8988
Train Data Columns: 18
Label Column: PM
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (199.1, 0.1, 8.83128, 7.8337)
	If 'regression' is n

Starting training fold 9.


Fitting 1 L1 models ...
Fitting model: LightGBM ...


[1000]	valid_set's l2: 8.21356	valid_set's r2: 0.80831


	0.8102	 = Validation score   (r2)
	2.52s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	0.8102	 = Validation score   (r2)
	0.0s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 3.05s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20220424_235518/")


In [9]:
RF = cv({'RF': {}})
RF.to_csv("Data/output/CV_random/RF.csv", index = False)

No path specified. Models will be saved in: "AutogluonModels/ag-20220425_000340/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20220425_000340/"
AutoGluon Version:  0.4.0
Python Version:     3.7.13
Operating System:   Linux
Train Data Rows:    8865
Train Data Columns: 18
Label Column: PM
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (199.1, 0.1, 8.82962, 7.78289)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11766.61 MB
	Train Data (Original)  Memory Usage: 1.28 MB (0.0% of available memory)
	Inferring data type o

10
Starting training fold 0.


	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	Types of features in processed data (raw dtype, special dtypes):
		('float', [])     : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])       :  1 | ['Day']
		('int', ['bool']) :  4 | ['Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	0.1s = Fit runtime
	18 features in original data used to generate 18 features in processed data.
	Train Data (Processed) Memory Usage: 1.03 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.1s ...
AutoGluon will gauge predictive performance using evaluation metric: 'r2'
	To change this, specify 

Starting training fold 1.


	0.7074	 = Validation score   (r2)
	12.93s	 = Training   runtime
	0.2s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	0.7074	 = Validation score   (r2)
	0.0s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 14.66s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20220425_000356/")
No path specified. Models will be saved in: "AutogluonModels/ag-20220425_000411/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20220425_000411/"
AutoGluon Version:  0.4.0
Python Version:     3.7.13
Operating System:   Linux
Train Data Rows:    8876
Train Data Columns: 18
Label Column: PM
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (199.1, 0.1, 8.80362, 7.64351)
	If 'regression' is

...

Fitting 1 L1 models ...
Fitting model: RandomForest ...
	0.6866	 = Validation score   (r2)
	12.83s	 = Training   runtime
	0.2s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	0.6866	 = Validation score   (r2)
	0.01s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 14.97s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20220425_000529/")
No path specified. Models will be saved in: "AutogluonModels/ag-20220425_000544/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20220425_000544/"
AutoGluon Version:  0.4.0
Python Version:     3.7.13
Operating System:   Linux
Train Data Rows:    8945
Train Data Columns: 18
Label Column: PM
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, st

Starting training fold 8.


	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	Types of features in processed data (raw dtype, special dtypes):
		('float', [])     : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])       :  1 | ['Day']
		('int', ['bool']) :  4 | ['Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	0.1s = Fit runtime
	18 features in original data used to generate 18 features in processed data.
	Train Data (Processed) Memory Usage: 1.04 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.11s ...
AutoGluon will gauge predictive performance using evaluation metric: 'r2'
	To change this, specify the eval_metric parameter of Predictor()
Automatically g

Starting training fold 9.


Fitting 1 L1 models ...
Fitting model: RandomForest ...
	0.8563	 = Validation score   (r2)
	13.05s	 = Training   runtime
	0.2s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	0.8563	 = Validation score   (r2)
	0.0s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 14.86s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20220425_000600/")


In [10]:
XT = cv({'XT': {}})
XT.to_csv("Data/output/CV_random/XT.csv", index = False)

No path specified. Models will be saved in: "AutogluonModels/ag-20220425_001218/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20220425_001218/"
AutoGluon Version:  0.4.0
Python Version:     3.7.13
Operating System:   Linux
Train Data Rows:    8866
Train Data Columns: 18
Label Column: PM
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (199.1, 0.1, 8.79383, 7.7627)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11745.8 MB
	Train Data (Original)  Memory Usage: 1.28 MB (0.0% of available memory)
	Inferring data type of 

10
Starting training fold 0.


			Note: Converting 4 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	Types of features in processed data (raw dtype, special dtypes):
		('float', [])     : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])       :  1 | ['Day']
		('int', ['bool']) :  4 | ['Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	0.1s = Fit runtime
	18 features in original data used to generate 18 features in processed data.
	Train Data (Processed) Memory Usage: 1.03 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.12s ...
AutoGluon wi

Starting training fold 1.


Fitting 1 L1 models ...
Fitting model: ExtraTrees ...
	0.8255	 = Validation score   (r2)
	4.04s	 = Training   runtime
	0.2s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	0.8255	 = Validation score   (r2)
	0.0s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 5.76s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20220425_001225/")
No path specified. Models will be saved in: "AutogluonModels/ag-20220425_001231/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20220425_001231/"
AutoGluon Version:  0.4.0
Python Version:     3.7.13
Operating System:   Linux
Train Data Rows:    8891
Train Data Columns: 18
Label Column: PM
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev)

...

Automatically generating train/validation split with holdout_frac=0.1, Train Rows: 8033, Val Rows: 893
Fitting 1 L1 models ...
Fitting model: ExtraTrees ...
	0.6347	 = Validation score   (r2)
	4.56s	 = Training   runtime
	0.2s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	0.6347	 = Validation score   (r2)
	0.01s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 6.72s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20220425_001307/")
No path specified. Models will be saved in: "AutogluonModels/ag-20220425_001314/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20220425_001314/"
AutoGluon Version:  0.4.0
Python Version:     3.7.13
Operating System:   Linux


Starting training fold 8.


Train Data Rows:    8942
Train Data Columns: 18
Label Column: PM
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (199.1, 0.1, 8.82815, 7.77626)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11745.18 MB
	Train Data (Original)  Memory Usage: 1.29 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 4 features to boolean dtype as they only contain 2 unique values.
	Sta

Starting training fold 9.


	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 4 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	Types of features in processed data (raw dtype, special dtypes):
		('float', [])     : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])       :  1 | ['Day']
		('int', ['bool']) :  4 | ['Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	0.1s = Fit runtime
	18 features in original data used to generate 18 features in processed data.
	Train Data (Processed) Memory Usage: 1.04 MB (0.0% of available memory)
Data preprocessing

In [11]:
KNN = cv({'KNN': {}})
KNN.to_csv("Data/output/CV_random/KNN.csv", index = False)

No path specified. Models will be saved in: "AutogluonModels/ag-20220425_002239/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20220425_002239/"
AutoGluon Version:  0.4.0
Python Version:     3.7.13
Operating System:   Linux
Train Data Rows:    8873
Train Data Columns: 18
Label Column: PM
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (199.1, 0.1, 8.82078, 7.78391)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11747.93 MB
	Train Data (Original)  Memory Usage: 1.28 MB (0.0% of available memory)
	Inferring data type o

10
Starting training fold 0.


	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	Types of features in processed data (raw dtype, special dtypes):
		('float', [])     : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])       :  1 | ['Day']
		('int', ['bool']) :  4 | ['Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	0.1s = Fit runtime
	18 features in original data used to generate 18 features in processed data.
	Train Data (Processed) Memory Usage: 1.03 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.11s ...
AutoGluon will gauge predictive performance using evaluation metric: 'r2'
	To change this, specify

Starting training fold 1.


	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	Types of features in processed data (raw dtype, special dtypes):
		('float', [])     : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])       :  1 | ['Day']
		('int', ['bool']) :  4 | ['Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	0.1s = Fit runtime
	18 features in original data used to generate 18 features in processed data.
	Train Data (Processed) Memory Usage: 1.03 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.1s ...
AutoGluon will gauge predictive performance using evaluation metric: 'r2'
	To change this, specify 

...

Fitting 1 L1 models ...
Fitting model: KNeighbors ...
	0.1071	 = Validation score   (r2)
	0.02s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	0.1071	 = Validation score   (r2)
	0.0s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 0.47s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20220425_002244/")
No path specified. Models will be saved in: "AutogluonModels/ag-20220425_002245/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20220425_002245/"
AutoGluon Version:  0.4.0
Python Version:     3.7.13
Operating System:   Linux
Train Data Rows:    8946
Train Data Columns: 18
Label Column: PM
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev)

Starting training fold 8.


	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 4 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	Types of features in processed data (raw dtype, special dtypes):
		('float', [])     : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])       :  1 | ['Day']
		('int', ['bool']) :  4 | ['Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	0.1s = Fit runtime
	18 features in original data used to generate 18 features in processed data.
	Train Data (Processed) Memory Usage: 1.04 MB (0.0% of available memory)
Data preprocessing

Starting training fold 9.


Fitting 1 L1 models ...
Fitting model: KNeighbors ...
	0.113	 = Validation score   (r2)
	0.02s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	0.113	 = Validation score   (r2)
	0.0s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 0.51s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20220425_002246/")


In [12]:
AutoGluon = cv('default')
AutoGluon.to_csv("Data/output/CV_random/AutoGluon.csv", index = False)

No path specified. Models will be saved in: "AutogluonModels/ag-20220425_002306/"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20220425_002306/"
AutoGluon Version:  0.4.0
Python Version:     3.7.13
Operating System:   Linux
Train Data Rows:    8852
Train Data Columns: 18
Label Column: PM
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (199.1, 0.1, 8.868, 7.89958)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11744.15 MB
	Train Data (Original)  Memory Usage: 1.27 MB (0.0% of available memory)
	Inferring data type of 

10
Starting training fold 0.


			Note: Converting 4 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])   :  5 | ['Day', 'Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	Types of features in processed data (raw dtype, special dtypes):
		('float', [])     : 13 | ['Lat', 'Lon', 'Elevation', 'Emissions', 'Roads', ...]
		('int', [])       :  1 | ['Day']
		('int', ['bool']) :  4 | ['Forest', 'Plumes_High', 'Plumes_Med', 'Plumes_Low']
	0.1s = Fit runtime
	18 features in original data used to generate 18 features in processed data.
	Train Data (Processed) Memory Usage: 1.03 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.1s ...
AutoGluon wil

[1000]	valid_set's l2: 20.811	valid_set's r2: 0.791245
[2000]	valid_set's l2: 20.3251	valid_set's r2: 0.795398
[3000]	valid_set's l2: 20.1436	valid_set's r2: 0.798475


	0.7988	 = Validation score   (r2)
	6.18s	 = Training   runtime
	0.17s	 = Validation runtime
Fitting model: LightGBM ...


...

	0.8162	 = Validation score   (r2)
	6.97s	 = Training   runtime
	0.19s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's l2: 6.84598	valid_set's r2: 0.845693


	0.8476	 = Validation score   (r2)
	2.85s	 = Training   runtime
	0.07s	 = Validation runtime
Fitting model: RandomForestMSE ...
	0.7881	 = Validation score   (r2)
	12.23s	 = Training   runtime
	0.2s	 = Validation runtime
Fitting model: CatBoost ...
	0.8637	 = Validation score   (r2)
	124.34s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesMSE ...
	0.7663	 = Validation score   (r2)
	3.92s	 = Training   runtime
	0.21s	 = Validation runtime
Fitting model: NeuralNetFastAI ...
	0.6329	 = Validation score   (r2)
	7.34s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: XGBoost ...
	0.8491	 = Validation score   (r2)
	14.01s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	0.598	 = Validation score   (r2)
	33.6s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: LightGBMLarge ...


[1000]	valid_set's l2: 6.7624	valid_set's r2: 0.845632


	0.8492	 = Validation score   (r2)
	5.67s	 = Training   runtime
	0.11s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	0.8708	 = Validation score   (r2)
	0.34s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 217.85s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20220425_005424/")


# Consolidate dataframes

In [13]:
small_ids = pd.read_csv("Data/datasets/Small_Ids.csv")['x']
small_ids = [float(i) for i in small_ids]

full_df = pd.DataFrame(columns = ['Model', 'test_area', 'station', 'Day', 'PM', 'PM_pred'])

paths = glob.glob("Data/output/CV_random/*.csv")

for path in paths:
  f = os.path.split(path)[1]
  Model = os.path.splitext(f)[0]
  Table = pd.read_csv(path)
  Table['test_area'] = Table['station'].isin(small_ids)
  Table['Model'] = Model

  full_df = full_df.append(Table, ignore_index = True)

full_df.to_csv("Data/output/all_cv_preds_random.csv", index = False)